In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding, LSTM
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.datasets import mnist, reuters
from keras.utils import np_utils, to_categorical

%matplotlib inline

Using TensorFlow backend.


In [5]:
docs = [
    """블로그보고 연락드립니다 !
맛집리뷰 같은 일반적인 홍보포스팅 용도로 일정기간 임대해주실수 있으실까요 ?4
기간따라 230부터 500까지 선지급 드립니다.

블로그외 계정이용은 원래대로 가능하시며
추후 재계약또한 선택가능하십니다.

불법적인 포스팅올라갈경우 계약금반환없이 계약파기또한 가능하시구요

원고받아서 직접올려주시고 건당나눠받는 알바형식도 가능하세요

관심있으시다면 아래 카톡으로 연락주세요

카톡ID
sansan94
""",
    """블로그보고 연락드려요6
맛집리뷰 또는 정보글 포스팅용도로 일정기간 대여해주시는조건으로
기간에 따라 최소230부터 500까지 선지급 드립니다.

대여 불가하신경우 원고받아서 직접올려주시고 건당으로 나눠받는 알바형식도 가능하세요

문의는 카톡으로 부탁드립니다.

( 블로그품질이 낮거나, 테스트 떨어진 경험이 있는 블로그라도 구하고있으니 편히 연락주세요 )

카톡ID : mks37""",
    """글 작성권한 관리 및 직접건당 부업 안내
210~400 무조건 먼저드려요
자세한건 툑 주세용!

일반적인 생활 정보 제공 글 작성용도로 사용
이전에 작성한 모든글 유지
부적절한 글 제공시 100% 자동파기 계약금 반환의무 없음
모든 앱기능 사용가능하고 체결서 먼저 작성


자세한 진행 절차는 아래로 상담 요청 부탁드립니다!

http://pf.kakao.com/_GxbFuK

캿캿툑 coco이십구 숫자""",
    """블로그 기능 일정기간 글 쓰기 권한 공유 문의

일상 생활에서 필요한 교육, 육아, 로펌, 등으로 이전 글 그대로 유지하고
새로운 폴더만 개설하여 한가지의 분야로만 업로드합니다.
불피요한 어긋나는 부분은 취급조차 하지않습니다!!

저와 함께해주시면 당일 최소 1"8'0 ~ 사백 까지 모두 먼저 드립니다!
절대 먼저 그무엇도 요구하지않습니다!

***
그외로 저희가 문제없는 완성된 글과 이미지를 보내드립니다.
받으신 자료를 그대로 올려주시면 익일 건당 지급!

자세한 부분은 아래 확인해주세요!
쭉 읽어보시고 생각있으시거나 궁금하신건 언제든지 편하게
채팅하기 꾹 눌러주세요! 저와 바로 연결 됩니다!

http://pf.kakao.com/_VuNwK
노랑이뚁 채널 미라둥이 검색!
(aaoaa하나육) -숫자입니다
""",
    """정해진 기한동안 글쓰기 작성권한을 공유받고싶습니다.
기간에따라 180~400까지 당일 선지급 약속합니다.

일상 로펌 건강 교육 등의 정보나 생활 리뷰글을 쓰는 용도로만 사용이 되며
이외 계정이용은 원래대로 가능하시며,
이전 글 모두 그대로 유지해드리는 조건입니다
불필요한건 절대 취급하지 않습니다!


완성된 글과 이미지를 받아 그대로 올리시고 건당수익 받는 형식도 가능하세요!

상세한건 아래를 참고해주세요!
http://pf.kakao.com/_GxbFuK
꺄캬옷뚁 c오c오299 (오는 알파벳입니다)""",
    """안녕하세요_1

블로그보고 연락드렸습니니다!
맛집, 일상, it, 뷰티 등의 리뷰또는 정보글을 포스팅하는 용도로
일정기간 활성화 관리대여를 요청드려봅니다:)

대여 기간에 따라 최소 180부터 최대 400까지 선지급 해드린뒤에
진행이 되고있습니다!

기간동안 블로그외에 계정이용은 원래대로 사용가능하십니다_

원고직접 받아서 올려주시는 건당으로 소액 나눠받는 형식도 진행가능 하십니다_

아래로 카톡 부탁드리겠습니다!
coco299
""",
    """안녕하세요3
일정기간 블로그 임대요청드리고자 연락드렸습니다.

일상 it 맛집 등의 리뷰나 정보글을 포스팅하는 용도로 일정기간 블로그 임대해주시는조건이며,

기간에따라서 최소180부터 400까지 선지급해드립니다.

원고받아서 직접올려주시고 건당 소액으로 나눠서받는형식도 가능하시구요

진행의사있으신경우 아래로 카톡연락부탁드립니다.
door334""",
    """반갑습니다.
블로그관련해서 일정기간 대여를요청드리고자 연락드려요5

일반적인 맛집이나 일상 등의 리뷰나 정보글을 포스팅하는용도로 일정기간 대여해주시는조건으로,
기간에따라서 최소180부터 400까지 선지급해드리고있습니다.

기간동안 블로그외 계정이용 원래대로 사용가능하시며,
불법적이거나 허위사실의 포스팅이올라갈경우 계약금반환없이 계약파기또한 가능하십니다.

원고드리면 받아서 직접올리는 건당알바도 가능

카톡 qetu687로 연락주세요 !""",
    """안녕하세요

맛집, it, 여행 등의 정보글또는 리뷰글 포스팅용도로 일정기간 블로그 대여요청드리고자 연락드려요7
기간에 따라 최소180부터 400까지 선지급 드립니다.

블로그외에 계정이용은 모두 가능하시고
추후 재계약또한 선택가능하십니다.

불법적인 포스팅이 올라갈경우 계약금반환없이 계약파기또한 가능하시구요

원고드리면 직접 포스팅해주시고 소액건당 받는형식도가능

관심있으시면 아래로 카톡부탁드립니다.
qetu687""",
    """안녕하세요

맛집, it, 여행 등의 정보글또는 리뷰글 포스팅용도로 일정기간 블로그 대여요청드리고자 연락드려요7
기간에 따라 최소180부터 400까지 선지급 드립니다.

블로그외에 계정이용은 모두 가능하시고
추후 재계약또한 선택가능하십니다.

불법적인 포스팅이 올라갈경우 계약금반환없이 계약파기또한 가능하시구요

원고드리면 직접 포스팅해주시고 소액건당 받는형식도가능

관심있으시면 아래로 카톡부탁드립니다.
qetu687""",
    """블로그보고 연락드려요8
맛집리뷰 또는 정보글 포스팅용도로 일정기간 대여해주시는조건으로
기간에 따라 최소180부터 400까지 선지급 드립니다.

기간동안 블로그외 계정이용 모두 가능하시고,
추후 재계약또한 선택가능하십니다.

대여기간동안 불법적이거나 허위포스팅이 올라갈경우 계약금반환없이 계약파기또한 가능하시구요

불가하신경우 원고받아서 직접올려주시고 건당나눠받는 알바형식도 가능하세요

관심있으시다면 아래로 톡부탁드립니다.
qetu687
""",
    """블로그보고 연락드립니다 !
맛집리뷰 같은 일반적 홍보포스팅 용도로 일정기간 임대해주실수 있으실까요 ?6
기간따라 180부터 400까지 선지급 드립니다.

블로그외 계정이용은 원래대로 가능하시며
추후 재계약또한 선택가능하십니다.

불법적인 포스팅올라갈경우 계약금반환없이 계약파기또한 가능하시구요

원고받아서 직접올려주시고 건당나눠받는 알바형식도 가능하세요

관심있으신경우 아래로 톡부탁드립니다.
qetu687""",
    """안녕하세요4
블로그관련 대여문의드리고자 연락드립니다.

블로그에서 쉽게볼수있는 맛집, 일상, 여행 등의
리뷰또는 정보글 포스팅하는 용도로 일정기간 대여를 요청드립니다.

해주시는 기간에따라서 최소180부터 400까지 선지급해드린뒤 진행됩니다.

기간동안 저희가 블로그포스팅용도로 사용케되며 그외에 계정이용은 원래대로 가능하십니다.

원고받아서 올리시는 건당알바도 가능합니다.

kkf347로 카톡부탁드려요 !""",
    """블로그 250 에 매입 합니다 카톡 BMTT 문의 ! ♽오늘의 명언♎ 큰 나무도 가느다란 가지에서 시작되는 것이다. 10층의 탑도 작은 벽돌을 하나씩 쌓아올리는 데에서 시작되는 것이며, 마지막에 이르기까지 처음과 마찬가지로 주의를 기울이면 어떤 일도 해낼 수 있을 것이다. - 노자""",
    """기자 단 가능하시면 확인 부탁 드립니다. 자세한 점 신청서 작성 후 보내주시면 답변 드리겠습니다. 감사하겠습니다. WWW.BIT.LY/2Lk0P8H 하도낙서""",
    """[♥기자단으루 영향력있어 혹시 고정진행 가능할까요? https://forms.gle/zmU2yxQJLHWWzBFQ8 연락 주시면 안내 드립니다 언제든지 기다리고 있겠습니다 차도살인""",
    """안녕하세요^^ B.l.o.g 200입니다. ㄲㅐ-톡 yesyes0724@nate.com""",
    """안녕하세요!^^　o병o원o홍o보o를　전문적으로하는　o광o고o회사입니다.　서이추이유는　블로그를　매입하기위해서이구요.　매입가는　100만원입니다.간단한　노출테스트후　통과되시면　바로매입(입금)　가능하시구요.　카톡아이디:　bmtt 　상담가능하시구요　　[수신거부-카카오톡 　bmtt　으로　아이디와　수신거부신청　해주시면　바로금지　금지리스트에　올리겠습니다.]　언제나행복한　하루되시구요~　잠들어 있는　블로그.　환영합니다~!""",
    """안녕하세요~　저희는　블로그　양도를　통해　정상병의원　홍보　블로그를　전문적으로　운영하고　있는　회사입니다.　애정어린　정성으로　관리해오신　블로그의　양도를　여쭙게　되어　죄송해요.　다만　더이상　시간여유가　없거나　여러가지　이유로　운영이　힘드시다면　이번　기회에　테스트　한　번　받아보시는게　어떠신가요?　매입금액은　50만원부터입니다.　이렇게　서이추부터　해서　불편하게　해드린　점　다시　한　번　사과드리고,　수신거부를　원하신다면　카톡으로　아이디와　수신거부라고　보내주시면　그　즉시　서이추　차단처리　하겠습니다.　카톡　ID:　blog8282　그럼　오늘도　좋은　하루　되세요!""",
    """안녕하세요~　저희는　블로그　양도를　통해　정상병의원　홍보　블로그를　전문적으로　운영하고　있는　회사입니다.　애정어린　정성으로　관리해오신　블로그의　양도를　여쭙게　되어　죄송해요.　다만　더이상　시간여유가　없거나　여러가지　이유로　운영이　힘드시다면　이번　기회에　테스트　한번　받아보시는게　어떠신가요?　매입금액은　50만원　이상　부터입니다.　이렇게　서이추부터　해서　불편하게　해드린　점　다시　한　번　사과드리고,　수신거부를　원하신다면　카톡으로　아이디와　수신거부라고　보내주시면　그　즉시　서이추　차단처리　하겠습니다.　카톡　ID:　blog8282　그럼　오늘도　좋은　하루　되세요!""",
    """안녕하세요 온라인광고 전문회사입니다. 지금 블로그를 통해서 병원과 학원 광고를 하고 있습니다. 이로인해 어느정도 관리가 된 블로그를 구매하고 있습니다. 구매가격은 40~45만원에 구매하고 있습니다. 블로그 판매에 대해 관심이 있으시다거나 궁금하신 점이 있으시다면 ◆연락처 : 010-6416-4001 카톡아이디 : mrqaz81 ◆ 연락주시면 성심성의껏 답변해드리겠습니다. 연락주세요^^"""
]

In [6]:
token = Tokenizer()
token.fit_on_texts(docs)
token.word_index

{'일정기간': 1,
 '드립니다': 2,
 '400까지': 3,
 '선지급': 4,
 '안녕하세요': 5,
 '계정이용은': 6,
 '아래로': 7,
 '블로그': 8,
 '등의': 9,
 '원래대로': 10,
 '계약금반환없이': 11,
 '계약파기또한': 12,
 '가능하시구요': 13,
 '원고받아서': 14,
 '글': 15,
 '일상': 16,
 '최소180부터': 17,
 '블로그보고': 18,
 '용도로': 19,
 '추후': 20,
 '재계약또한': 21,
 '선택가능하십니다': 22,
 '직접올려주시고': 23,
 '가능하세요': 24,
 '기간에': 25,
 '따라': 26,
 '모두': 27,
 '맛집': 28,
 '맛집리뷰': 29,
 '블로그외': 30,
 '불법적인': 31,
 '알바형식도': 32,
 '연락주세요': 33,
 '포스팅용도로': 34,
 'com': 35,
 '그대로': 36,
 'it': 37,
 '기간동안': 38,
 'qetu687': 39,
 '연락드립니다': 40,
 '일반적인': 41,
 '가능하시며': 42,
 '건당나눠받는': 43,
 '정보글': 44,
 '대여해주시는조건으로': 45,
 '먼저': 46,
 '자세한': 47,
 'http': 48,
 'pf': 49,
 'kakao': 50,
 '정보글을': 51,
 '포스팅하는': 52,
 '블로그외에': 53,
 '카톡': 54,
 '기간에따라서': 55,
 '원고드리면': 56,
 '여행': 57,
 '가능하시고': 58,
 '올라갈경우': 59,
 '있습니다': 60,
 '같은': 61,
 '홍보포스팅': 62,
 '임대해주실수': 63,
 '있으실까요': 64,
 '기간따라': 65,
 '500까지': 66,
 '포스팅올라갈경우': 67,
 '관심있으시다면': 68,
 '아래': 69,
 '카톡으로': 70,
 '카톡id': 71,
 '또는': 72,
 '대여': 73,
 '불가하신경우': 74,
 '건당으로': 75,
 '나눠받는': 76,

In [18]:
token.index_word.get(1)

'일정기간'

In [15]:
token.word_counts.get('일정기간')

13

In [19]:
for key in token.index_word.keys():
    print(f'#{key} {token.index_word.get(key)}: {token.word_counts.get(token.index_word.get(key))}')

#1 일정기간: 13
#2 드립니다: 9
#3 400까지: 9
#4 선지급: 8
#5 안녕하세요: 8
#6 계정이용은: 7
#7 아래로: 7
#8 블로그: 7
#9 등의: 7
#10 원래대로: 6
#11 계약금반환없이: 6
#12 계약파기또한: 6
#13 가능하시구요: 6
#14 원고받아서: 6
#15 글: 6
#16 일상: 6
#17 최소180부터: 6
#18 블로그보고: 5
#19 용도로: 5
#20 추후: 5
#21 재계약또한: 5
#22 선택가능하십니다: 5
#23 직접올려주시고: 5
#24 가능하세요: 5
#25 기간에: 5
#26 따라: 5
#27 모두: 5
#28 맛집: 5
#29 맛집리뷰: 4
#30 블로그외: 4
#31 불법적인: 4
#32 알바형식도: 4
#33 연락주세요: 4
#34 포스팅용도로: 4
#35 com: 4
#36 그대로: 4
#37 it: 4
#38 기간동안: 4
#39 qetu687: 4
#40 연락드립니다: 3
#41 일반적인: 3
#42 가능하시며: 3
#43 건당나눠받는: 3
#44 정보글: 3
#45 대여해주시는조건으로: 3
#46 먼저: 3
#47 자세한: 3
#48 http: 3
#49 pf: 3
#50 kakao: 3
#51 정보글을: 3
#52 포스팅하는: 3
#53 블로그외에: 3
#54 카톡: 3
#55 기간에따라서: 3
#56 원고드리면: 3
#57 여행: 3
#58 가능하시고: 3
#59 올라갈경우: 3
#60 있습니다: 3
#61 같은: 2
#62 홍보포스팅: 2
#63 임대해주실수: 2
#64 있으실까요: 2
#65 기간따라: 2
#66 500까지: 2
#67 포스팅올라갈경우: 2
#68 관심있으시다면: 2
#69 아래: 2
#70 카톡으로: 2
#71 카톡id: 2
#72 또는: 2
#73 대여: 2
#74 불가하신경우: 2
#75 건당으로: 2
#76 나눠받는: 2
#77 부탁드립니다: 2
#78 안내: 2
#79 생활: 2
#80 작성: 2
#81 gxbfuk: 2
#82 문의: 2
#83 교육